# Navigation

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
from unityagents import UnityEnvironment
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# please do not modify the line below
env = UnityEnvironment(file_name='/usr/src/app/Banana_Linux_NoVis/Banana.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [2]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [3]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agent while it is training**, and you should set `train_mode=True` to restart the environment.

In [4]:
# env_info = env.reset(train_mode=True)[brain_name] # reset the environment
# state = env_info.vector_observations[0]            # get the current state
# score = 0                                          # initialize the score
# while True:
#     action = np.random.randint(action_size)        # select an action
#     env_info = env.step(action)[brain_name]        # send the action to the environment
#     next_state = env_info.vector_observations[0]   # get the next state
#     reward = env_info.rewards[0]                   # get the reward
#     done = env_info.local_done[0]                  # see if episode has finished
#     score += reward                                # update the score
#     state = next_state                             # roll over the state to next time step
#     if done:                                       # exit loop if episode finished
#         break
    
# print("Score: {}".format(score))

When finished, you can close the environment.

In [5]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agent while it is training.  However, **_after training the agent_**, you can download the saved model weights to watch the agent on your own machine! 

In [6]:
from collections import namedtuple, deque

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import random

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
class DuelingNN(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=400, fc2_units=300): 
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
            fc3_units (int): Number of nodes in third hidden layer
        """
        super(DuelingNN, self).__init__()
        self.seed = torch.manual_seed(seed)
        
        self.feature = nn.Sequential(
                        nn.Linear(state_size, fc1_units),
                        nn.ReLU()
                        )
        
        self.advantage = nn.Sequential(
                        nn.Linear(fc1_units, fc2_units),
                        nn.ReLU(),
                        nn.Linear(fc2_units, action_size)
                        )

        self.value = nn.Sequential(
                        nn.Linear(fc1_units, fc2_units),
                        nn.ReLU(),
                        nn.Linear(fc2_units, 1)
                        )
        


    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = self.feature(state)
        
        advantage = self.advantage(x)
        value     = self.value(x)
        
        return value + advantage  - advantage.mean()



In [9]:
class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, config, state_size, action_size, seed):
        """Initialize an Agent object.
        
        Params
        ======
            config (dict): constants for the agent
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """
        self.config = config
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Q-Network
        self.qnetwork_local = DuelingNN(state_size, action_size, seed).to(device)
        self.qnetwork_target = DuelingNN(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=self.config['LR'])

        # Replay memory
        self.memory = PrioritizedReplayBuffer(action_size, self.config['BUFFER_SIZE'], 
                                              self.config['BATCH_SIZE'], self.config['PROB_ALPHA'],seed)

        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done, step):
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, done, 1.0)
        
        # Learn every UPDATE_EVERY time steps.
        if len(self.memory) > self.config['BATCH_SIZE'] and step % self.config['UPDATE_FREQUENCY'] == 0:
            for _ in range(self.config['UPDATE_TIMES']):
                indices, experiences = self.memory.sample()
                self.learn(experiences, indices, self.config['GAMMA'])

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()

        with torch.no_grad():
            action_values = self.qnetwork_local(state)

        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, indices, gamma):
        """Update value parameters using given batch of experience tuples.

        Params
        ======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones, weights = experiences

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # Compute Q targets for current states 
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)

        prios  = (Q_expected - Q_targets).pow(2) * weights
        prios += self.config['TD_ERROR_CONSTANT']
        self.memory.update_priorities(indices, prios)
        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(self.qnetwork_local, self.qnetwork_target, self.config['TAU'])                     

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

In [10]:
class PrioritizedReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, prob_alpha, seed):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.prob_alpha = prob_alpha
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done", "priority"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done, weight):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done, weight)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        prios = np.array([e.priority for e in self.memory])
        
        probs = prios ** self.prob_alpha
        probs /= probs.sum()
        
        indices = np.random.choice(len(self.memory), self.batch_size, p=probs, replace=False)
        experiences = [e for i, e in enumerate(self.memory) if i in indices]
        
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        priorities = torch.from_numpy(np.vstack([e.priority for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        
        return indices, (states, actions, rewards, next_states, dones, priorities)

    def update_priorities(self, indices, prios):
        "update weights of memory"
        for idx, priority in zip(indices, prios):
            self.memory[idx]._replace(priority=priority)
    
    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

In [11]:
def deep_q_nn(n_episodes=2000, eps_start=1.0, eps_end=0.01, eps_decay=0.995, n_score=100):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                      
    scores_window = deque(maxlen=n_score)  
    eps = eps_start                  
    
    for i_episode in range(1, n_episodes+1):
        i_step = 0
        env_info = env.reset(train_mode=True)[brain_name] 
        state = env_info.vector_observations[0]            
        score = 0
        while True:            
            action = int(agent.act(state, eps))                 
            env_info = env.step(action)[brain_name]        
            next_state = env_info.vector_observations[0]   
            reward = env_info.rewards[0]                   
            done = env_info.local_done[0]                  
            agent.step(state, action, reward, next_state, done, i_step) 
            state = next_state                             
            score += reward
            i_step += 1
            if done:
                break 
        scores_window.append(score)       
        scores.append(score)              
        eps = max(eps_end, eps_decay*eps) # decrease epsilon

        # Print Score while training
        print('\rEpisode {}\tAverage Score: {:.2f}\tCurrent Score: {:.2f}'.format(i_episode, np.mean(scores_window), score), end="")
        if i_episode % n_score == 0:
            
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))

        if np.mean(scores_window) >= 13.:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'model_rainbow.pth') 
            break
    return scores

In [ ]:
# constants for the agent
agent_config = {
'BUFFER_SIZE': int(1e5),  # replay buffer size
'BATCH_SIZE': 64,         # minibatch size
'GAMMA': 0.99,            # discount factor
'TAU': 1e-3,              # for soft update of target parameters
'LR': 1e-3,               # learning rate 
"UPDATE_FREQUENCY": 4,    # how long to wait to update the network
"UPDATE_TIMES": 1,    # how often to update the network
"PROB_ALPHA": 0,    #  probability alpha for priotization
"TD_ERROR_CONSTANT": 1e-5    # adds to TD error for prioreplaybuffer to prevent zeros  
}

agent = Agent(agent_config, state_size, action_size,seed=0)

scores = deep_q_nn()

Episode 100	Average Score: 0.42	Current Score: 2.000
Episode 153	Average Score: 1.30	Current Score: 3.000

In [ ]:
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

fig.tight_layout()
fig.savefig("plot.jpg")

In [ ]:
# load the weights from file
# agent.qnetwork_local.load_state_dict(torch.load('model_rainbow.pth'))

# for i in range(3):
#    state = env.reset()
#    img = plt.imshow(env.render(mode='rgb_array'))
#    for j in range(200):
#        action = agent.act(state)
#        img.set_data(env.render(mode='rgb_array')) 
#        plt.axis('off')
#        display.display(plt.gcf())
#        display.clear_output(wait=True)
#        state, reward, done, _ = env.step(action)
#        if done:
#            break 
#            
#env.close()